#### Observations :
The top 3 scores obtained were :
1. 0.14366
2. 0.14945
3. 0.15538
All the scores have relu  activation with Adam Optimizer and MSE loss.

The first score was obtained by using 210 epochs, learning rate=0.01 and 2100 neurtons in the hidden layer
The second score was obtained by using 200 epochs, learning rate =0.001 and 2000 neurons in the hidden layer
The third score was obtained by 150 epochs, learning rate 0.001 and 2000 neurons in the hidden layer.

As the number of neurons and epochs increased, the network has been trained well and hence the score has increased.Also, with the top score architecture, when I added a batch size of 128 to the data, I got a reduced score. Noticed that the normalization reduced the score , so didnt perform normalization.

![alt text](kaggle_result.png "Title")

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
from sklearn import preprocessing


In [2]:
def trainingModel(dataLoader):
    for epoch in range(210):
        ave_loss=0.0
        print("Epoch ",epoch+1)
        for i,data in enumerate(dataLoader,0):
            inputs,target =data
            target = target.float()
            optimizer.zero_grad()
            output=model(inputs)
            loss=criteria(output,target)
            loss.backward()
            optimizer.step()
            ave_loss+=loss.item()
        print("Loss for Epoch ",epoch+1," is ",(ave_loss/x_train.shape[0]))

In [3]:
def dataPreProcessing(dataframe):
    print("Number of NAN columns before preprocessing\n",dataframe.isna().sum().sort_values()[-30:-1]) # prints the top 35 nan valued columns
    
    #Filling with "NA" string
    for col in ['Alley','FireplaceQu','Fence','MiscFeature','PoolQC','GarageQual','GarageFinish','GarageType','GarageCond','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2','BsmtExposure']:
        dataframe[col].fillna('NA', inplace=True)
    #Fill numerical columns with  mode value
    fill_avg = ['KitchenQual','Exterior1st','SaleType','Exterior2nd','Functional','Utilities','MSZoning','LotFrontage','Electrical','MasVnrArea','GarageYrBlt']
    for i in fill_avg:
        dataframe[i].fillna(dataframe[i].value_counts().to_frame().index[0], inplace=True)
    
    median_values_col=['GarageArea','GarageCars','BsmtFinSF1','TotalBsmtSF','BsmtFinSF2','BsmtFullBath','BsmtHalfBath','BsmtUnfSF']
    for i in median_values_col: # did it separately as the test data had nan in it.. didnt find it at training dataframe
        dataframe[i].fillna(dataframe[i].value_counts().to_frame().index[0], inplace=True)
        
    print("Number of NAN columns after preprocessing\n",dataframe.isna().sum().sort_values()[-10:-1])
    
    return dataframe
    

In [4]:
# Perform one-hot encoding of the categorical columns in the dataframe
def performOnehotEncoding(dataframe):
    col_groups = dataframe.columns.to_series().groupby(dataframe.dtypes).groups
    non_numeric_cols = col_groups[np.dtype('O')]
    for col in non_numeric_cols:
        one_hot = pd.get_dummies(dataframe[col],prefix=str(col+"_"))
        dataframe = dataframe.drop(col,axis=1)
        dataframe = dataframe.join(one_hot)
    return dataframe


In [5]:
# this function gets the columns not in test but present in train and adds in the
# index present in the train dataframe
def addColumnsFromTraintoTest(train,test):
    for i in train.columns:
        if i not in test.columns:
            index= train.columns.get_loc(i)
            test.insert(index,i,0)
    return test
    

In [6]:
#change the dataframe to numpy array to tensor array
def changeToTensor(dataFrame):
    x = dataFrame.values
    x= x.astype('float64')
    x= torch.Tensor(x)
    return x

In [7]:
train = pd.read_csv("house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")

In [8]:
train.shape, test.shape

((1460, 81), (1459, 80))

In [9]:
train = dataPreProcessing(train)

Number of NAN columns before preprocessing
 YearBuilt          0
RoofStyle          0
RoofMatl           0
Exterior1st        0
Exterior2nd        0
ExterQual          0
ExterCond          0
Foundation         0
MSSubClass         0
YearRemodAdd       0
Utilities          0
Electrical         1
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
BsmtFinType2      38
BsmtExposure      38
GarageQual        81
GarageFinish      81
GarageYrBlt       81
GarageType        81
GarageCond        81
LotFrontage      259
FireplaceQu      690
Fence           1179
Alley           1369
MiscFeature     1406
dtype: int64
Number of NAN columns after preprocessing
 ExterCond      0
ExterQual      0
MasVnrArea     0
Exterior2nd    0
Exterior1st    0
RoofMatl       0
RoofStyle      0
Heating        0
SalePrice      0
dtype: int64


In [10]:
test = dataPreProcessing(test)

Number of NAN columns before preprocessing
 Exterior1st        1
SaleType           1
TotalBsmtSF        1
BsmtUnfSF          1
Exterior2nd        1
BsmtFinSF1         1
BsmtFinSF2         1
BsmtFullBath       2
Functional         2
Utilities          2
BsmtHalfBath       2
MSZoning           4
MasVnrArea        15
MasVnrType        16
BsmtFinType2      42
BsmtFinType1      42
BsmtExposure      44
BsmtQual          44
BsmtCond          45
GarageType        76
GarageFinish      78
GarageYrBlt       78
GarageQual        78
GarageCond        78
LotFrontage      227
FireplaceQu      730
Fence           1169
Alley           1352
MiscFeature     1408
dtype: int64
Number of NAN columns after preprocessing
 ExterCond        0
ExterQual        0
MasVnrArea       0
Exterior2nd      0
Exterior1st      0
RoofMatl         0
RoofStyle        0
SaleType         0
SaleCondition    0
dtype: int64


In [11]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000


In [12]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NA,Reg,Lvl,AllPub,...,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NA,IR1,Lvl,AllPub,...,0,0,NA,NA,NA,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NA,IR1,HLS,AllPub,...,144,0,NA,NA,NA,0,1,2010,WD,Normal


In [13]:
train_num= train.values
test_num= test.values
labels=train_num[:,-1]
train.drop(['SalePrice','Id'],axis=1, inplace=True)
test.drop('Id',axis=1, inplace=True)

In [14]:
train = performOnehotEncoding(train)

In [15]:
test = performOnehotEncoding(test)

In [16]:
test = addColumnsFromTraintoTest(train,test)

In [17]:
x_train = changeToTensor(train)
labels= labels.astype('float64')
labels= torch.Tensor(labels)
x_test = changeToTensor(test)

In [18]:
class Net(nn.Module):
    def __init__(self,size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(size,2100)
        self.fc6 = nn.Linear(2100,1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc6(x)
        return x

In [19]:
x_train.shape, labels.shape

(torch.Size([1460, 302]), torch.Size([1460]))

In [20]:
x_test.shape

torch.Size([1459, 302])

In [21]:
dataset= utils.TensorDataset(x_train,labels)
dataLoader = utils.DataLoader(dataset)

In [22]:
model = Net(x_train.shape[1])

In [23]:
model

Net(
  (fc1): Linear(in_features=302, out_features=2100, bias=True)
  (fc6): Linear(in_features=2100, out_features=1, bias=True)
)

In [24]:
criteria= nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [25]:
trainingModel(dataLoader)

Epoch  1


/work/test/venv/lib/python3.5/site-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Loss for Epoch  1  is  4701269376.414348
Epoch  2
Loss for Epoch  2  is  2900069181.1582117
Epoch  3
Loss for Epoch  3  is  2451435120.5033584
Epoch  4
Loss for Epoch  4  is  2240945621.929738
Epoch  5
Loss for Epoch  5  is  2142103883.2658536
Epoch  6
Loss for Epoch  6  is  2385008599.4340467
Epoch  7
Loss for Epoch  7  is  1982733974.6796942
Epoch  8
Loss for Epoch  8  is  1947689995.3613453
Epoch  9
Loss for Epoch  9  is  1882199822.4821603
Epoch  10
Loss for Epoch  10  is  1852823603.5840547
Epoch  11
Loss for Epoch  11  is  1805629136.2024505
Epoch  12
Loss for Epoch  12  is  1727306753.4164557
Epoch  13
Loss for Epoch  13  is  1696984887.7612686
Epoch  14
Loss for Epoch  14  is  1610500092.956284
Epoch  15
Loss for Epoch  15  is  1557749979.3521786
Epoch  16
Loss for Epoch  16  is  1620064890.0085974
Epoch  17
Loss for Epoch  17  is  1501793979.1791956
Epoch  18
Loss for Epoch  18  is  1459644136.5274823
Epoch  19
Loss for Epoch  19  is  1425743313.510751
Epoch  20
Loss for Epoch

Loss for Epoch  157  is  646451250.8616719
Epoch  158
Loss for Epoch  158  is  601105027.6529005
Epoch  159
Loss for Epoch  159  is  648224697.4252856
Epoch  160
Loss for Epoch  160  is  620955230.3389566
Epoch  161
Loss for Epoch  161  is  627975658.4404733
Epoch  162
Loss for Epoch  162  is  666441997.7695056
Epoch  163
Loss for Epoch  163  is  594916058.4071345
Epoch  164
Loss for Epoch  164  is  624623725.847572
Epoch  165
Loss for Epoch  165  is  642603986.8416963
Epoch  166
Loss for Epoch  166  is  598387249.8665967
Epoch  167
Loss for Epoch  167  is  553002409.8782372
Epoch  168
Loss for Epoch  168  is  622728767.2816402
Epoch  169
Loss for Epoch  169  is  714890629.8582613
Epoch  170
Loss for Epoch  170  is  592683339.2741858
Epoch  171
Loss for Epoch  171  is  580109805.5751145
Epoch  172
Loss for Epoch  172  is  653584884.9587489
Epoch  173
Loss for Epoch  173  is  564652965.9958824
Epoch  174
Loss for Epoch  174  is  605403285.5218008
Epoch  175
Loss for Epoch  175  is  5942

In [26]:
predictions = model(x_test)

In [27]:
predictions_array = predictions.data.cpu().numpy() # convert the tensor to numpy array

In [28]:
# set the 
test_data = pd.read_csv("house-prices-advanced-regression-techniques/test.csv")
test_df=pd.DataFrame(columns=['Id','SalePrice'])
test_df["Id"]= test_data["Id"]
test_df["SalePrice"]= predictions_array
test_df[["Id","SalePrice"]].to_csv("submission.csv",index=False)